In [2]:
# ! pip install python-dotenv

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

# Loading Credentials
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
api_endpoint = os.getenv("AZURE_BASE_URL")
together_api = os.getenv("TOGETHER_API_KEY")
chat_model = os.getenv("CHAT_MODEL")
embedding_model = os.getenv("EMBEDDING_MODEL")

In [12]:
# ! pip install openai
# ! pip install langchain langchain_community

### Langchain model initialization

In [44]:
from langchain.chat_models import AzureChatOpenAI


llm = AzureChatOpenAI(
    api_key=api_key, 
    model_name=chat_model,
    api_version = api_version,
    azure_endpoint = api_endpoint,
    temperature=0
    )

In [14]:
''' 
Prompt Creation
'''

from langchain.prompts import PromptTemplate

# Define a prompt template that instructs the model to analyze and expand on the topic
research_prompt = PromptTemplate(
    input_variables=["topic"],
    template=(
        '''
        You are a research assistant. The user has asked the following research question or provided this topic: '{topic}'.
        Please analyze the topic, explain it back to the user, and suggest 4 additional related questions and ideas that could help expand the research. 
        Generate a list of 5 search queries (including the original one) or topics that could be helpful in researching about the topic.
        ''' 
    )
)


In [15]:
from langchain import LLMChain
from langchain.chains import SimpleSequentialChain

# Create an LLMChain using the defined prompt template
research_chain = LLMChain(
    llm=llm,
    prompt=research_prompt
)

# Simple Sequential Chain to handle the processing
sequential_chain = SimpleSequentialChain(
    chains=[research_chain],
    verbose=True
)

/tmp/ipykernel_27505/3597435984.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  research_chain = LLMChain(


In [16]:
''' 
Sample Generation : Testing our initializations
'''

def generate_research_insights(topic):
    # Run the sequential chain with the provided topic
    response = sequential_chain.run(topic)
    return response

# Example usage
topic = "The impact of climate change on polar bear populations"
result = generate_research_insights(topic)
print(result)

/tmp/ipykernel_27505/4203453993.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = sequential_chain.run(topic)




> Entering new SimpleSequentialChain chain...
### Topic Analysis

The topic "The impact of climate change on polar bear populations" focuses on understanding how the changing climate, particularly in the Arctic regions, affects polar bears, their habitats, and their overall survival. Climate change leads to rising temperatures, melting ice caps, and altered ecosystems, which directly impact polar bears, as they rely on sea ice for hunting seals, their primary food source. As the ice diminishes, polar bears may face challenges such as reduced access to food, increased competition, and changes in their breeding patterns. This topic is crucial for conservation efforts and understanding the broader implications of climate change on biodiversity.

### Related Questions and Ideas

1. **How does the melting Arctic ice affect the hunting behavior and diet of polar bears?**
   - Investigating the specific changes in hunting patterns and dietary shifts due to ice loss.

2. **What are the long-

### We'll be using below resources to gather more information around our reserrch
1. YouTube
2. Semantic Scholar
3. Web Articles and Blogs

#### YouTube

In [21]:
# ! pip install --upgrade youtube_search

In [22]:
from langchain_community.tools import YouTubeSearchTool # this library has Python function for searching for youtube videosss

tool = YouTubeSearchTool()
tool.run(topic, 5) # 5 is the number we want the tool to return us from the avaiable videos

['https://www.youtube.com/watch?v=0TXbUt1j6Mc&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z0gcJCU8JAYcqIYzv', 'https://www.youtube.com/watch?v=V-YFOlwwe_g&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z']

"['https://www.youtube.com/watch?v=0TXbUt1j6Mc&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z0gcJCU8JAYcqIYzv', 'https://www.youtube.com/watch?v=V-YFOlwwe_g&pp=ygU2VGhlIGltcGFjdCBvZiBjbGltYXRlIGNoYW5nZSBvbiBwb2xhciBiZWFyIHBvcHVsYXRpb25z']"

#### Wikipedia

In [24]:
# ! pip install wikipedia
# ! pip install xmltodict # formating the output

In [27]:
''' 
Using only Wikipedia retriever
'''
from langchain_community.retrievers import WikipediaRetriever

# retriever cell
retriever = WikipediaRetriever()
retriever.invoke(topic)

[Document(metadata={'title': 'Polar bear', 'summary': "The polar bear (Ursus maritimus) is a large bear native to the Arctic and nearby areas. It is closely related to the brown bear, and the two species can interbreed. The polar bear is the largest extant species of bear and land carnivore, with adult males weighing 300–800 kg (660–1,760 lb). The species is sexually dimorphic, as adult females are much smaller. The polar bear is white- or yellowish-furred with black skin and a thick layer of fat. It is more slender than the brown bear, with a narrower skull, longer neck and lower shoulder hump. Its teeth are sharper and more adapted to cutting meat. The paws are large and allow the bear to walk on ice and paddle in the water.\nPolar bears are both terrestrial and pagophilic (ice-living) and are considered marine mammals because of their dependence on marine ecosystems. They prefer the annual sea ice but live on land when the ice melts in the summer. They are mostly carnivorous and spe

In [ ]:
''' 
Using Agents to retrieve relevant information from Wikipedia
'''
from langchain.agents import AgentType, initialize_agent
from langchain.agents import load_tools

# Wikipedia tool agent
tools = load_tools(
                    ["wikipedia"], 
                    llm=llm
                    )

agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

agent(topic)

#### Semantic Scholar
Semantic Scholar uses natural language and machine learning to find relevant results across all disciplines, including biology, medicine, computer science, geography, business, history, and economics. It includes a scientific literature graph and billions of citations, and its corpus contains nearly 200 million academic papers from publisher partnerships, data providers, and web crawls.

In [ ]:
# ! pip install --upgrade --quiet  semanticscholar

In [38]:
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.utilities.semanticscholar import SemanticScholarAPIWrapper

api_wrapper = SemanticScholarAPIWrapper(doc_content_chars_max=1000, top_k_results=5)
tools = [SemanticScholarQueryRun(api_wrapper=api_wrapper)]

In [45]:
''' 
Here for Semantic Scholar we're creating a Prompt to retrive more contectual information. In above case for Wikipedia we've not created any prompt if you check in tools.
'''

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert researcher.",
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_MULTI_FUNCTIONS,
    prompt=prompt,
    verbose=True
)

In [46]:
response = agent_executor.invoke(
    {
        "input": f"""query: {topic}, for each of the relevant paper that you find :Create 3 different sections: List of papers with url for it, summary of that paper and citations in that paper, do this for each paper   """
    }
)



> Entering new AgentExecutor chain...

Invoking: `semanticscholar` with `{'query': 'The impact of climate change on polar bear populations'}`


Published year: 2016
Title: Modelling the Impact of Climate Change on the Polar Bear Population in Western Hudson Bay
Authors: Nicole Bastow
Abstract: None

Invoking: `semanticscholar` with `{'query': 'The impact of climate change on polar bear populations'}`


Published year: 2016
Title: Modelling the Impact of Climate Change on the Polar Bear Population in Western Hudson Bay
Authors: Nicole Bastow
Abstract: None
I found a relevant paper on the impact of climate change on polar bear populations. Here are the details:

### 1. List of Papers
- **Title**: Modelling the Impact of Climate Change on the Polar Bear Population in Western Hudson Bay
- **Authors**: Nicole Bastow
- **Published Year**: 2016
- **URL**: [Link to Paper](https://www.semanticscholar.org/paper/Modelling-the-Impact-of-Climate-Change-on-the-Bastow/)

### 2. Summary of the Paper

#### Playwright Browser

- Web Scraping: Automatically extract data from web pages, such as product prices, reviews, or contact information.
- Web Testing: Automate testing of web applications by simulating user interactions and verifying expected outcomes.
- Research: Gather information from various websites on a specific topic and summarize the findings.
- Personal Automation: Automate tasks that involve interacting with web applications, such as checking email, managing social media, or booking appointments.

In [53]:
''' 
Install below using Terminal
'''

# ! pip install playwright beautifulsoup4
# ! playwright install

' \nInstall below using Terminal\n'

In [72]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit

from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",      },
)
import nest_asyncio

nest_asyncio.apply()

In [73]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

PydanticUserError: `PlayWrightBrowserToolkit` is not fully defined; you should define `SyncBrowser`, then call `PlayWrightBrowserToolkit.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.10/u/class-not-fully-defined